In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [52]:
# using pandas get dummies method to turn caticorgical data to numeric and create x & y
stevens = pd.read_csv('/Users/michaelcolellajensen/Desktop/Data Sets/Stevens.csv')
stevens_binary = pd.get_dummies(stevens[['Circuit', 'Issue', 'Petitioner', 'Respondent', 'LowerCourt', 'Unconst']], 
drop_first=True)

x=stevens_binary.values
y=stevens['Reverse'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=123)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 46 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   Unconst                                 566 non-null    int64
 1   Circuit_11th                            566 non-null    uint8
 2   Circuit_1st                             566 non-null    uint8
 3   Circuit_2nd                             566 non-null    uint8
 4   Circuit_3rd                             566 non-null    uint8
 5   Circuit_4th                             566 non-null    uint8
 6   Circuit_5th                             566 non-null    uint8
 7   Circuit_6th                             566 non-null    uint8
 8   Circuit_7th                             566 non-null    uint8
 9   Circuit_8th                             566 non-null    uint8
 10  Circuit_9th                             566 non-null    uint8
 11  Circuit_DC         

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [64]:
# building a DecisionTreeClassifier untuned
dt_model = DecisionTreeClassifier(max_depth=4, random_state=123, criterion='gini')
dt_model.fit(x_train, y_train)
dt_predictions = dt_model.predict(x_test)
dt_proba = dt_model.predict_proba(x_test)[:,1]

dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:{:.3f}".format(dt_accuracy))
dt_roc = roc_auc_score(y_test, dt_proba)
print("Decision Tree AUC score:{:.3f}".format(dt_roc))

dt_matrix = confusion_matrix(y_test, dt_predictions)
print('Confusion Matrix\n', dt_matrix)

Decision Tree Accuracy:0.675
Decision Tree AUC score:0.727
Confusion Matrix
 [[25 25]
 [12 52]]


In [92]:
# building a tuned DecisionTreeClassifier
dt_params = {'max_depth':[2,3,4],
    'min_samples_leaf': [0.05, 0.09, 0.13],
    'max_features': [0.2, 0.4, 0.6, 0.8],
    }
grid_dt = GridSearchCV(estimator=dt_model, 
    param_grid = dt_params,
    scoring = 'accuracy',
    cv=5 
    )
grid_dt.fit(x_train, y_train)
dt_best_params = grid_dt.best_params_
print('Best hyperparamerters:\n', dt_best_params)

best_dt_model = grid_dt.best_estimator_
best_dt_model_predictions = best_dt_model.predict(x_test)
best_dt_proba = best_dt_model.predict_proba(x_test)[:,1]
best_dt_auc = roc_auc_score(y_test, best_dt_proba)
best_dt_accuracy = accuracy_score(y_test, best_dt_model_predictions)
print('Best Decision Tree Accuracy:{:.3f}'.format(best_dt_accuracy))
print('Best Decision Tree AUC Score:{:.3f}'.format(best_dt_auc))

best_dt_matrix = confusion_matrix(y_test, best_dt_model_predictions)
print('Best Decision Tree Matrix:\n', best_dt_matrix)


Best hyperparamerters:
 {'max_depth': 2, 'max_features': 0.2, 'min_samples_leaf': 0.13}
Best Decision Tree Accuracy:0.728
Best Decision Tree AUC Score:0.700
Best Decision Tree Matrix:
 [[32 18]
 [13 51]]


In [78]:
# building an untuned RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, 
    max_depth = 6, 
    min_samples_leaf = 0.1, 
    random_state = 123)

rf_model.fit(x_train, y_train)
rf_predictions = rf_model.predict(x_test)
rf_proba = rf_model.predict_log_proba(x_test)[:,1]

rf_accuracy = accuracy_score(y_test, rf_predictions)
print('Random Forest Accuracy:{:.3f}'.format(rf_accuracy))

rf_auc = roc_auc_score(y_test, rf_proba)
print('Random Forest AUC Score:{:.3f}'.format(rf_auc))

rf_matrix = confusion_matrix(y_test, rf_predictions)
print('Random Forest Confusion Matrix:\n', rf_matrix)

Random Forest Accuracy:0.596
Random Forest AUC Score:0.706
Random Forest Confusion Matrix:
 [[ 6 44]
 [ 2 62]]


In [90]:
# building a RandomForestClassifier with tuning
print(rf_model.get_params())
rf_params = {
    'n_estimators':[90, 100, 110],
    'max_depth': [3, 4, 5], 
    'max_features': ['sqrt', 'auto'],
    'min_samples_leaf': [0.06, 0.07, 0.08]
    }
rf_grid = GridSearchCV(estimator=rf_model,
    param_grid = rf_params,
    cv=5,
    scoring = 'accuracy',
    verbose = 1,
    n_jobs = -1)
rf_grid.fit(x_train, y_train)
best_rf_params = rf_grid.best_params_
print('Best Random Forest Hyperparamers:\n',(best_rf_params))

best_rf_model = rf_grid.best_estimator_
best_rf_predictions = best_rf_model.predict(x_test)
best_rf_proba = best_rf_model.predict_proba(x_test)[:,1]
best_rf_accuracy = accuracy_score(y_test, best_rf_predictions)
best_rf_auc = roc_auc_score(y_test, best_rf_proba)

print('Best Random Forest Accuracy:{:.3f}'.format(best_rf_accuracy))
print('Best Random Forest AUC Score:{:.3f}'.format(best_rf_auc))

best_rf_matrix = confusion_matrix(y_test, best_rf_predictions)
print('Best Random Forest Confusion Matrix:\n', best_rf_matrix)


{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 0.1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False}
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Random Forest Hyperparamers:
 {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 0.07, 'n_estimators': 100}
Best Random Forest Accuracy:0.693
Best Random Forest AUC Score:0.725
Best Random Forest Confusion Matrix:
 [[21 29]
 [ 6 58]]
